In [1]:
import torch
import os
import matplotlib as plt
import numpy as np
import pandas as pd
import sys
import random
from helpy_train import *
import helpy_log
from torch.utils.data import Dataset, DataLoader
import gensim.downloader
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from dataset import *
from mlp import *

punks = string.punctuation
punks = punks+ '``'+ "''"
stopword_list = stopwords.words('english')


%load_ext autoreload
%autoreload 2
torch.manual_seed(88)
np.random.seed(88)
random.seed(88)

In [2]:
def trial(batch_size_in, learning_rate_in, momentum_in, weight_decay_in, save_folder):
    print(f'save_folder:{save_folder}')
    
    device = f"cuda" if torch.cuda.is_available() else "cpu"
    
    tr_loader,va_loader,te_loader  = get_train_val_test_loaders(batch_size_in)
    
    model = MLP()
    model.to(device)

    
    start_epoch = 0
    stats = []
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate_in, weight_decay=weight_decay_in)

    #axes,fig = helpy_log.make_training_plot(batch_size_in, learning_rate_in, momentum_in, weight_decay_in)

    saved_path = os.path.join(save_folder,
                              f"b{batch_size_in}_lr{learning_rate_in}_p{momentum_in}_wd{weight_decay_in}")
    info = {"batch":batch_size_in, "lr":learning_rate_in,"p": momentum_in, "wd": weight_decay_in}
    
    if not os.path.exists(saved_path):
        os.makedirs(saved_path, exist_ok=True)
    
    
    print("inital eval")
    evaluate_epoch(tr_loader, va_loader, te_loader, model, criterion, start_epoch, stats,
                   device,info, save_folder)


    global_min_loss = stats[-1][-2]
    
    patience = 5
    curr_count_to_patience = 0
    
    # Loop over the entire dataset multiple times
    epoch = start_epoch
    print(f"Entering train loop for lr:{learning_rate_in} p:{momentum_in} wd:{weight_decay_in}")
    while curr_count_to_patience < patience:
        print(f"starting epoch {epoch}")
        
        # Train model
        train_epoch(tr_loader, model, criterion, optimizer, device)

        # Evaluate model
        evaluate_epoch(tr_loader, va_loader, te_loader,model, criterion, epoch + 1, stats,
                       device, info, save_folder)

        # Save model parameters
        save_checkpoint(model, epoch + 1, save_folder, stats, info)

        if epoch > 8:
            curr_count_to_patience, global_min_loss = early_stopping(stats, curr_count_to_patience, global_min_loss)
        epoch += 1
    print(f"Finished Training after {epoch} epochs")

In [4]:
trial(64, 1e-2, 0.8, 1e-4,'results')

save_folder:results
inital eval
Epoch:0
train loss 0.8379999995231628
val loss 0.8199999928474426
test loss 0.8259999752044678

Entering train loop for lr:0.01 p:0.8 wd:0.0001
starting epoch 0
Epoch:1
train loss 0.40400001406669617
val loss 0.4059999883174896
test loss 0.40299999713897705

starting epoch 1
Epoch:2
train loss 0.39399999380111694
val loss 0.39800000190734863
test loss 0.39800000190734863

starting epoch 2
